[![stars](https://img.shields.io/github/stars/laminlabs/lamindb?logo=GitHub&color=yellow)](https://github.com/laminlabs/lamindb)
[![pypi](https://img.shields.io/pypi/v/lamindb?color=blue&label=pypi%20package)](https://pypi.org/project/lamindb)
[![cran](https://www.r-pkg.org/badges/version/laminr?color=green)](https://cran.r-project.org/package=laminr)

# Introduction

LaminDB is an open-source data framework for better computational biology.
It lets you track data transformations, curate datasets, manage metadata, and query a built-in database for common biological data types.

:::{dropdown} Why?

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/BunYmHkyFLITlM5MYQck.svg" width="350px" style="background: transparent" align="right">

Reproducing analytical results or understanding how a dataset or model was created can be a pain.
Leave alone training models on historical data, orthogonal assays, or datasets generated by other teams.

Biological datasets are typically managed with versioned storage systems (file systems, object storage, git, dvc), {term}`UI`-focused community or SaaS platforms, structureless data lakes, rigid data warehouses (SQL, monolithic arrays), and data lakehouses for tabular data.

LaminDB goes beyond these systems with a lakehouse that models biological datasets beyond tables with enough structure to enable queries and enough freedom to keep the pace of R&D high.

For data structures like `DataFrame`, `AnnData`, `.zarr`, `.tiledbsoma`, etc., LaminDB tracks and provides the rich context that collaborative biological research requires:

- data lineage: data sources and transformations; scientists and machine learning models
- domain knowledge and experimental metadata: the features and labels derived from domain entities

In this [blog post](https://lamin.ai/blog/problems), we discuss a breadth of data management problems of the field.

:::

:::{dropdown} LaminDB specs

```{include} includes/features-lamindb.md
```
:::

LaminHub is a data collaboration hub built on LaminDB similar to how GitHub is built on git.

:::{dropdown} LaminHub overview

```{include} includes/features-laminhub.md
```
:::

## Quickstart

Install the `lamindb` Python package.

```shell
pip install 'lamindb[jupyter,bionty]'  # support notebooks & biological ontologies
```

Connect to a LaminDB instance.

```shell
lamin connect account/instance  # <-- replace with your instance
```

Access an input dataset and save an output dataset.

::::{tab-set}
:::{tab-item} Python
```python
import lamindb as ln
	
ln.track()  # track a run of your notebook or script
artifact = ln.Artifact.get("3TNCsZZcnIBv2WGb0001")  # get an artifact by uid
filepath = artifact.cache()  # cache the artifact on disk

# do your work

ln.Artifact("./my_dataset.csv", key="my_results/my_dataset.csv").save()  # save a file
ln.finish()  # mark the run as finished & save a report for the current notebook/script
```
:::
:::{tab-item} R

```R
# laminr needs pip install 'lamindb'
install.packages("laminr", dependencies = TRUE)  # install the laminr package from CRAN
library(laminr)

db <- connect()  # connect to the instance you configured on the terminal
db$track()  # track a run of your notebook or script
artifact <- db$Artifact$get("3TNCsZZcnIBv2WGb0001")  # get an artifact by uid
filepath <- artifact$cache()  # cache the artifact on disk

# do your work

db$Artifact.from_path("./my_dataset.csv", key="my_results/my_dataset.csv").save()  # save a folder
db$finish()  # mark the run finished
```

Depending on whether you ran RStudio's notebook mode, you may need to save an html export for `.qmd` or `.Rmd` file via the command-line.

```shell
lamin save my-analysis.Rmd
```

For more, see the [R docs](https://laminr.lamin.ai/).

:::
::::

## Walkthrough

A LaminDB instance is a database that manages metadata for datasets in different storage locations. Let's create one.

In [ ]:
!lamin init --storage ./lamin-intro --modules bionty

You can also pass a cloud storage location to `--storage` (S3, GCP, R2, HF, etc.) or a Postgres database connection string to `--db`. See {doc}`setup`.
If you decide to connect your LaminDB instance to the hub, you will see data & metadata in a UI.

```{dropdown} On the hub.

<a href="https://lamin.ai/laminlabs/lamindata">
<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/YuefPQlAfeHcQvtq0000.png" width="700px">
</a>

```

### Transforms

A data transformation (a "transform") is a piece of code (script, notebook, pipeline, function) that can be applied to input data to produce output data.

When you call {meth}`~lamindb.track` in a script or notebook, inputs, outputs, source code, run reports and environments start to get automatically tracked.

In [ ]:
import lamindb as ln
import pandas as pd

ln.track()  # track the current notebook or script

:::{dropdown} Is this compliant with OpenLineage?

Yes. What OpenLineage calls a "job", LaminDB calls a "transform". What OpenLineage calls a "run", LaminDB calls a "run".

:::

You can see all your transforms and their runs in the {class}`~lamindb.Transform` and {class}`~lamindb.Run` registries.

In [ ]:
ln.Transform.df()

In [ ]:
ln.Run.df()

### Artifacts

An {class}`~lamindb.Artifact` stores a dataset or model as a file or folder.

In [ ]:
# an example dataset
df = ln.core.datasets.small_dataset1(otype="DataFrame", with_typo=True)
df

In [ ]:
# create & save an artifact from a DataFrame
artifact = ln.Artifact.from_df(df, key="my_datasets/rnaseq1.parquet").save()

# describe the artifact
artifact.describe()

Copy or download the artifact into a local cache.

In [ ]:
artifact.cache()

Open the artifact for streaming.

In [ ]:
dataset = artifact.open()  # returns pyarrow.Dataset
dataset.head(2).to_pandas()

Cache & load the artifact into memory.

In [ ]:
artifact.load()

View data lineage.

In [ ]:
artifact.view_lineage()

:::{dropdown} How do I create an artifact for a file or folder?

Source path is local:

```python
ln.Artifact("./my_data.fcs", key="my_data.fcs")
ln.Artifact("./my_images/", key="my_images")
```
<br>

Upon `artifact.save()`, the source path will be copied or uploaded into your instance's current storage, visible & changeable via `ln.settings.storage`.

If the source path is remote or already in a registered storage location, `artifact.save()` won't trigger a copy or upload but register the existing path.

```python
ln.Artifact("s3://my-bucket/my_data.fcs")  # key is auto-populated from S3, you can optionally pass a description
ln.Artifact("s3://my-bucket/my_images/")  # key is auto-populated from S3, you can optionally pass a description
```
<br>
You can also use other remote file systems supported by `fsspec`.

:::

```{dropdown} How does LaminDB compare to a AWS S3?

LaminDB provides a database on top of AWS S3 (or GCP storage, file systems, etc.).

Similar to organizing files with paths, you can organize artifacts using the `key` parameter of {class}`~lamindb.Artifact`.

However, you'll see that you can more conveniently query data by entities you care about: people, code, experiments, genes, proteins, cell types, etc.

```

:::{dropdown} Are artifacts aware of array-like data?

Yes.

You can make artifacts from paths referencing array-like objects:

```python
ln.Artifact("./my_anndata.h5ad", key="my_anndata.h5ad")
ln.Artifact("./my_zarr_array/", key="my_zarr_array")
```

Or from in-memory objects:

```python
ln.Artifact.from_df(df, key="my_dataframe.parquet")
ln.Artifact.from_anndata(adata, key="my_anndata.h5ad")
```

You can open large artifacts for slicing from the cloud or load small artifacts directly into memory.

:::

Just like transforms, artifacts are versioned. Let's create a new version by revising the dataset.

In [ ]:
# keep the dataframe with a typo around - we'll need it later
df_typo = df.copy()

# fix the "IFNJ" typo
df["perturbation"] = df["perturbation"].cat.rename_categories({"IFNJ": "IFNG"})

# create a new version
artifact = ln.Artifact.from_df(df, key="my_datasets/rnaseq1.parquet").save()

# see all versions of an artifact
artifact.versions.df()

:::{dropdown} Can I also create new versions independent of `key`?

That works, too, you can use `revises`:

```python
artifact_v1 = ln.Artifact.from_df(df, description="Just a description").save()
# below revises artifact_v1
artifact_v2 = ln.Artifact.from_df(df_updated, revises=artifact_v1).save()
```

<br>

The good thing about passing `revises: Artifact` is that you don't need to worry about coming up with naming conventions for paths.

The good thing about versioning based on `key` is that it's how all data versioning tools are doing it.

:::

### Labels

Annotate an artifact with a {class}`~lamindb.ULabel` and a {class}`bionty.CellType`. The same works for any entity in any custom schema module.

In [ ]:
import bionty as bt

# create & save a typed label
experiment_type = ln.ULabel(name="Experiment", is_type=True).save()
candidate_marker_experiment = ln.ULabel(
    name="Candidate marker experiment", type=experiment_type
).save()

# label the artifact
artifact.ulabels.add(candidate_marker_experiment)

# repeat for a bionty entity
cell_type = bt.CellType.from_source(name="effector T cell").save()
artifact.cell_types.add(cell_type)

# describe the artifact
artifact.describe()

For annotating datasets with parsed labels like the cell_mediums `DMSO` & `IFNG`, jump to "Curate datasets".

### Registries

LaminDB's central classes are registries that store records ({class}`~lamindb.core.Record` objects).

The easiest way to see the latest records for a registry is to call the _class method_ {class}`~lamindb.core.Record.df`.

In [ ]:
ln.ULabel.df()

A record and its registry share the same fields, which define the metadata you can query for. If you want to see them, look at the class or auto-complete.

In [ ]:
ln.Artifact

### Query & search

You can write arbitrary relational queries using the class methods {class}`~lamindb.core.Record.get` and {class}`~lamindb.core.Record.filter`.
The syntax for it is Django's query syntax.

In [ ]:
# get a single record (here the current notebook)
transform = ln.Transform.get(key="introduction.ipynb")

# get a set of records by filtering on description
ln.Artifact.filter(key__startswith="my_datasets/").df()

# query all artifacts ingested from a transform
artifacts = ln.Artifact.filter(transform=transform).all()

# query all artifacts ingested from a notebook with "intro" in the title and labeled "Candidate marker experiment"
artifacts = ln.Artifact.filter(
    transform__description__icontains="intro", ulabels=candidate_marker_experiment
).all()

The class methods {class}`~lamindb.core.Record.search` and {class}`~lamindb.core.Record.lookup` help with approximate matches.

In [ ]:
# search in a registry
ln.Transform.search("intro").df()

# look up records with auto-complete
ulabels = ln.ULabel.lookup()
cell_types = bt.CellType.lookup()

:::{dropdown} Show me a screenshot

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/lgRNHNtMxjU0y8nIagt7.png" width="400px">

:::

### Features

You can annotate datasets by associated features.

In [ ]:
# define the "temperature" & "experiment" features
ln.Feature(name="temperature", dtype=float).save()
ln.Feature(
    name="experiment", dtype=ln.ULabel
).save()  # categorical values are validated against the ULabel registry

# annotate
artifact.features.add_values(
    {"temperature": 21.6, "experiment": "Candidate marker experiment"}
)

# describe the artifact
artifact.describe()

Query artifacts by features.

In [ ]:
ln.Artifact.features.filter(experiment__contains="marker experiment").df()

The easiest way to validate & annotate a dataset by the features they measure is via a `Curator`: jump to "Curate datasets".

## Key use cases

### Understand data lineage

Understand where a dataset comes from and what it's used for ([background](inv:docs#project-flow)).

```python
artifact.view_lineage()
```

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/KQmzmmLOeBN0C8Ykitjn.svg" width="800">

:::{dropdown} I just want to see the transforms.

```python
transform.view_lineage()
```

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/b0geN1HDHXlORqMOOPay.svg" width="400">

:::

You don't need a workflow manager to track data lineage (if you want to use one, see {doc}`docs:pipelines`). All you need is:

```python
import lamindb as ln

ln.track()  # track your run, start tracking inputs & outputs

# your code

ln.finish()  # mark run as finished, save execution report, source code & environment
```

```{dropdown} On the hub.

Below is how a single transform ([a notebook](https://lamin.ai/laminlabs/lamindata/transform/PtTXoc0RbOIq65cN)) with its run report looks on the hub.

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/RGXj5wcAf7EAc6J8dJfH.png" width="900px">

```

To create a new version of a notebook or script, run `lamin load` on the terminal, e.g.,

```bash
$ lamin load https://lamin.ai/laminlabs/lamindata/transform/13VINnFk89PE0004
→ notebook is here: mcfarland_2020_preparation.ipynb
```

### Curate datasets

You already saw how to ingest datasets without validation.
This is often enough if you're prototyping or working with one-off studies.
But if you want to create a big body of standardized data, you have to invest the time to curate your datasets.

Let's define a {class}`~lamindb.Schema` to curate a `DataFrame`.

In [ ]:
# define valid labels
perturbation_type = ln.ULabel(name="Perturbation", is_type=True).save()
ln.ULabel(name="DMSO", type=perturbation_type).save()
ln.ULabel(name="IFNG", type=perturbation_type).save()

# define the schema
schema = ln.Schema(
    name="My DataFrame schema",
    features=[
        ln.Feature(name="ENSG00000153563", dtype=int).save(),
        ln.Feature(name="ENSG00000010610", dtype=int).save(),
        ln.Feature(name="ENSG00000170458", dtype=int).save(),
        ln.Feature(name="perturbation", dtype=ln.ULabel).save(),
    ],
).save()

With a `Curator`, we can save an _annotated_ & _validated_ artifact with a single line of code.

In [ ]:
curator = ln.curators.DataFrameCurator(df, schema)

# save curated artifact
artifact = curator.save_artifact(key="my_curated_dataset.parquet")  # calls .validate()

# see the parsed annotations
artifact.describe()

# query for a ulabel that was parsed from the dataset
ln.Artifact.get(ulabels__name="IFNG")

If we feed a dataset with an invalid dtype or typo, we'll get a `ValidationError`.

In [ ]:
curator = ln.curators.DataFrameCurator(df_typo, schema)

# validate the dataset
try:
    curator.validate()
except ln.errors.ValidationError as error:
    print(str(error))

### Manage biological registries

The generic {class}`~lamindb.Feature` and {class}`~lamindb.ULabel` registries will get you pretty far.

But let's now look at what you do can with a dedicated biological registry like {class}`~bionty.Gene`.

Every {py:mod}`bionty` registry is based on configurable public ontologies (>20 of them).

In [ ]:
cell_types = bt.CellType.public()
cell_types

In [ ]:
cell_types.search("gamma-delta T cell").head(2)

Define an `AnnData` schema.

In [ ]:
# define var schema
var_schema = ln.Schema(
    name="my_var_schema",
    itype=bt.Gene.ensembl_gene_id,
    dtype=int,
).save()

obs_schema = ln.Schema(
    name="my_obs_schema",
    features=[
        ln.Feature(name="perturbation", dtype=ln.ULabel).save(),
    ],
).save()

# define composite schema
anndata_schema = ln.Schema(
    name="my_anndata_schema",
    otype="AnnData",
    components={"obs": obs_schema, "var": var_schema},
).save()

Validate & annotate an `AnnData`.

In [ ]:
import anndata as ad
import bionty as bt

# store the dataset as an AnnData object to distinguish data from metadata
adata = ad.AnnData(
    df[["ENSG00000153563", "ENSG00000010610", "ENSG00000170458"]],
    obs=df[["perturbation"]],
)

# save curated artifact
curator = ln.curators.AnnDataCurator(adata, anndata_schema)
artifact = curator.save_artifact(description="my RNA-seq")
artifact.describe()

Query for typed features.

In [ ]:
# get a lookup object for human genes
genes = bt.Gene.filter(organism__name="human").lookup()
# query for all feature sets that contain CD8A
feature_sets = ln.FeatureSet.filter(genes=genes.cd8a).all()
# write the query
ln.Artifact.filter(feature_sets__in=feature_sets).df()

Update ontologies, e.g., create a cell type record and add a new cell state.

In [ ]:
# create an ontology-coupled cell type record and save it
neuron = bt.CellType.from_source(name="neuron").save()

# create a record to track a new cell state
new_cell_state = bt.CellType(
    name="my neuron cell state", description="explains X"
).save()

# express that it's a neuron state
new_cell_state.parents.add(neuron)

# view ontological hierarchy
new_cell_state.view_parents(distance=2)

### Scale learning

How do you integrate new datasets with your existing datasets? Leverage {class}`~lamindb.Collection`.

In [ ]:
# a new dataset
df2 = ln.core.datasets.small_dataset2(otype="DataFrame")
adata = ad.AnnData(
    df2[["ENSG00000153563", "ENSG00000010610", "ENSG00000004468"]],
    obs=df2[["perturbation"]],
)
curator = ln.curators.AnnDataCurator(adata, anndata_schema)
artifact2 = curator.save_artifact(key="my_datasets/my_rnaseq2.h5ad")

Create a collection using {class}`~lamindb.Collection`.

In [ ]:
collection = ln.Collection([artifact, artifact2], key="my-RNA-seq-collection").save()
collection.describe()
collection.view_lineage()

In [ ]:
# if it's small enough, you can load the entire collection into memory as if it was one
collection.load()

# typically, it's too big, hence, open it for streaming (if the backend allows it)
# collection.open()

# or iterate over its artifacts
collection.artifacts.all()

# or look at a DataFrame listing the artifacts
collection.artifacts.df()

Directly train models on collections of `AnnData`.

```
# to train models, batch iterate through the collection as if it was one array
from torch.utils.data import DataLoader, WeightedRandomSampler
dataset = collection.mapped(obs_keys=["cell_medium"])
sampler = WeightedRandomSampler(
    weights=dataset.get_label_weights("cell_medium"), num_samples=len(dataset)
)
data_loader = DataLoader(dataset, batch_size=2, sampler=sampler)
for batch in data_loader:
    pass
```

Read this [blog post](https://lamin.ai/blog/arrayloader-benchmarks) for more on training models on sharded datasets.

## Design

### World model

1. Teams need to have enough freedom to initiate work independently but enough structure to easily integrate datasets later on
2. Batched datasets ({class}`~lamindb.Artifact`) from physical instruments are transformed ({class}`~lamindb.Transform`) into useful representations
3. Learning needs features ({class}`~lamindb.Feature`, {class}`~bionty.CellMarker`, ...) and labels ({class}`~lamindb.ULabel`, {class}`~bionty.CellLine`, ...)
4. Insights connect dataset representations with experimental metadata and knowledge (ontologies)

### Architecture

LaminDB is a distributed system like git that can be run or hosted anywhere. As infrastructure, you merely need a database (SQLite/Postgres) and a storage location (file system, S3, GCP, HuggingFace, ...).

You can easily create your new local instance:

::::{tab-set}
:::{tab-item} Shell
```bash
lamin init --storage ./my-data-folder
```
:::
:::{tab-item} Python
```python
import lamindb as ln
ln.setup.init(storage="./my-data-folder")
```
:::
::::

Or you can let collaborators connect to a cloud-hosted instance:

::::{tab-set}
:::{tab-item} Shell
```bash
lamin connect account-handle/instance-name
```
:::
:::{tab-item} Python
```python
import lamindb as ln
ln.connect("account-handle/instance-name")
```
:::
:::{tab-item} R
```R
library(laminr)
ln <- connect("account-handle/instance-name")
```
:::
::::

For learning more about how to create & host LaminDB instances on distributed infrastructure, see {doc}`setup`. LaminDB instances work standalone but can optionally be managed by LaminHub. For an architecture diagram of LaminHub, [reach out](https://lamin.ai/contact)!


### Database schema & API

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/XoTQFCmmj2uU4d2xyj9u.png" width="350px" style="background: transparent" align="right">

LaminDB provides a SQL schema for common metadata entities: {class}`~lamindb.Artifact`, {class}`~lamindb.Collection`, {class}`~lamindb.Transform`, {class}`~lamindb.Feature`, {class}`~lamindb.ULabel` etc. - see the [API reference](/api) or the [source code](https://github.com/laminlabs/lnschema-core/blob/main/lnschema_core/models.py).

The core metadata schema is extendable through modules (see green vs. red entities in **graphic**), e.g., with basic biological ({class}`~bionty.Gene`, {class}`~bionty.Protein`, {class}`~bionty.CellLine`, etc.) & operational entities (`Biosample`, `Techsample`, `Treatment`, etc.).

```{dropdown} What is the metadata schema language?

Data models are defined in Python using the Django ORM. Django translates them to SQL tables.
[Django](https://github.com/django/django) is one of the most-used & highly-starred projects on GitHub (~1M dependents, ~73k stars) and has been robustly maintained for 15 years.

```

On top of the metadata schema, LaminDB is a Python API that models datasets as artifacts, abstracts over storage & database access, data transformations, and (biological) ontologies.

Note that the schemas of datasets (e.g., `.parquet` files, `.h5ad` arrays, etc.) are modeled through the `Feature` registry and do not require migrations to be updated.

### Custom registries

LaminDB can be extended with registry modules building on the [Django](https://github.com/django/django) ecosystem. Examples are:

- [bionty](./bionty): Registries for basic biological entities, coupled to public ontologies.
- [wetlab](https://github.com/laminlabs/wetlab): Registries for samples, treatments, etc.

If you'd like to create your own module:

1. Create a git repository with registries similar to [wetlab](https://github.com/laminlabs/wetlab)
2. Create & deploy migrations via `lamin migrate create` and `lamin migrate deploy`

### Repositories

LaminDB and its plugins consist in open-source Python libraries & publicly hosted metadata assets:

- [lamindb](https://github.com/laminlabs/lamindb): Core package.
- [bionty](https://github.com/laminlabs/bionty): Registries for basic biological entities, coupled to public ontologies.
- [wetlab](https://github.com/laminlabs/wetlab): Registries for samples, treatments, etc.
- [usecases](https://github.com/laminlabs/lamin-usecases): Use cases as visible on the docs.

All immediate dependencies are available as git submodules [here](https://github.com/laminlabs/lamindb/tree/main/sub), for instance,

- [lamindb-setup](https://github.com/laminlabs/lamindb-setup): Setup & configure LaminDB.
- [lamin-cli](https://github.com/laminlabs/lamin-cli): CLI for `lamindb` and `lamindb-setup`.

For a comprehensive list of open-sourced software, browse our [GitHub account](https://github.com/laminlabs).

- [lamin-utils](https://github.com/laminlabs/lamin-utils): Generic utilities, e.g., a logger.
- [readfcs](https://github.com/laminlabs/readfcs): FCS artifact reader.
- [nbproject](https://github.com/laminlabs/readfcs): Light-weight Jupyter notebook tracker.
- [bionty-assets](https://github.com/laminlabs/bionty-assets): Assets for public biological ontologies.

LaminHub is not open-sourced.

### Influences

LaminDB was influenced by many other projects, see {doc}`docs:influences`.